In [1]:
import os

In [2]:
%pwd

'/config/workspace/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/config/workspace'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [16]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [21]:
from MlProject.utils.common import read_yaml, create_directories
from MlProject.constants import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [18]:
import pandas as pd
import os
from MlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [19]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        x_train = train_data.drop([self.config.target_column],axis=1)
        x_test = test_data.drop([self.config.target_column],axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=7)
        lr.fit(x_train,y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [22]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise(e)

[2024-01-08 12:19:55,591: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-08 12:19:55,594: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-08 12:19:55,597: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-08 12:19:55,599: INFO: common: created directory at: artifacts]
[2024-01-08 12:19:55,600: INFO: common: created directory at: artifacts/model_trainer]
ModelTrainerConfig(root_dir='artifacts/model_trainer', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_name='model.joblib', alpha=0.2, l1_ratio=0.1, target_column='quality')
